# Running fastFMM directly in a IRkernel

In [1]:
library("fastFMM")
library("arrow") # need to use arrow for new feather format


Attaching package: ‘arrow’


The following object is masked from ‘package:utils’:

    timestamp




In [2]:
data <- read_feather("../../../trialexp_notebook_learning/xr_aborted.feather")

In [3]:
df <- as.data.frame(data)
head(df)

,trial_id,animal_id,diff_bar_off_time,DA.1,DA.2,DA.3,DA.4,DA.5,DA.6,DA.7,⋯,DA.343,DA.344,DA.345,DA.346,DA.347,DA.348,DA.349,DA.350,zscored_diff_time,__index_level_0__
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,0,RE017,311,-1.2571774,-2.0395348,-2.27895529,-1.312244610,-0.5420776,-0.68070898,-0.52327721,⋯,1.69788532,1.46719733,1.5514194,1.436470044,1.5035468,2.0196757,2.46773513,2.3395865,-0.4290805,0
2,1,RE017,171,-3.8612223,-3.8194726,-4.34583669,-4.723615997,-4.3172153,-4.02632673,-4.43120038,⋯,-1.56612276,-0.96724989,-0.3742277,-0.005861236,0.1260335,0.6067790,1.47703769,1.7935542,-0.5064940,1
3,2,RE017,-344,-0.2024804,-0.1865520,-0.11856224,-0.001694632,0.3250853,0.81185517,0.95068136,⋯,0.01048129,0.02372886,0.2425918,0.299588174,0.2880776,0.5532802,0.65785249,0.4498099,-0.7912650,2
4,3,RE017,455,0.2358278,0.1442528,0.25994757,0.665430019,0.5025797,-0.34661679,-0.72506561,⋯,1.66321792,1.11980858,1.4123681,1.787130852,1.8141190,1.7888551,1.79555070,1.7459085,-0.3494552,3
5,10,RE017,252,2.5403254,2.8537247,2.30023533,1.283489226,1.2353338,1.65030786,1.64207132,⋯,0.60953337,0.60828874,0.3206043,0.102328252,0.2624744,0.6683993,0.76149541,0.5120715,-0.4617048,10
6,11,RE017,-13,1.6027430,0.7743659,-0.08128942,0.311422222,0.6267507,-0.01063283,-0.06754593,⋯,0.09755075,-0.19274938,-0.2336273,-0.300684788,-0.4326520,-0.1241173,0.05704814,-0.4709049,-0.6082374,11


In [9]:
dat = read.csv("binary.csv") # read in data
head(as.matrix(round(dat[,1:10], 3)))

id,session,trial,cs,photometry.1,photometry.2,photometry.3,photometry.4,photometry.5,photometry.6
3,1,1,0,-0.810,-0.626,-0.484,-0.414,-0.336,-0.335
3,1,2,0,-0.037,-0.136,0.005,-0.022,0.029,0.251
3,1,3,0,-0.241,-0.058,0.037,-0.075,-0.242,-0.276
3,1,4,0,0.058,0.032,-0.041,0.080,0.062,-0.047
3,1,5,0,-0.134,0.024,0.017,-0.128,-0.108,-0.086
3,1,6,0,-0.547,-0.547,-0.444,-0.380,-0.309,-0.329


In [5]:
mod = fui(DA ~ zscored_diff_time + (1 | animal_id), data = df, parallel = TRUE) # fit random slope model


Number of cores used for parallelization: 21



In [6]:
model_results = plot_fui(mod,  return = TRUE)

ERROR: Error: object 'mod' not found


In [13]:
type(model_results)

ERROR: Error in type(model_results): could not find function "type"


In [15]:
library(ncdf4)

In [16]:
nc <- nc_open("sresa1b_ncar_ccsm3-example.nc")

In [25]:
pr <- nc$var[['pr']]

# use tidync

In [27]:
ice_file <- system.file("extdata", "ifremer", "20171002.nc", package = "tidync", mustWork = TRUE)

In [40]:
library(tidync)
nc <-tidync(ice_file)

In [41]:
nc


Data Source (1): 20171002.nc ...

Grids (2) <dimension family> : <associated variables> 

[1]   D0,D1,D2 : concentration, quality_flag    **ACTIVE GRID** ( 419648  values per variable)
[2]   D2       : time

Dimensions 3 (all active): 
  
  dim   name  length     min     max start count    dmin    dmax unlim coord_dim 
  <chr> <chr>  <dbl>   <dbl>   <dbl> <int> <int>   <dbl>   <dbl> <lgl> <lgl>     
1 D0    ni       632       1     632     1   632       1     632 FALSE FALSE     
2 D1    nj       664       1     664     1   664       1     664 FALSE FALSE     
3 D2    time       1 1032204 1032204     1     1 1032204 1032204 FALSE TRUE      

In [31]:
hf <- concentration %>% 
  hyper_filter(ni = index > 150, 
               nj = dplyr::between(index, 30, 100))

In [33]:
hf %>% hyper_array()

Class: tidync_data (list of tidync data arrays)
Variables (2): 'concentration', 'quality_flag'
Dimension (2): ni,nj,time (482, 71)
Source: /home/MRC.OX.AC.UK/ndcn1330/R/x86_64-pc-linux-gnu-library/4.5/tidync/extdata/ifremer/20171002.nc

In [44]:
df <- nc %>%
  hyper_tibble(select_var = c("concentration"))

In [45]:
df

concentration,ni,nj
<int>,<chr>,<chr>
0,291,5
0,292,5
0,293,5
0,294,5
0,295,5
0,296,5
0,297,5
0,298,5
0,299,5


## Load xarray file

In [8]:
library(tidync)
xarray_nc <- tidync("../../../trialexp_notebook_learning/xr_aborted.nc")
xarray_nc


Data Source (1): xr_aborted.nc ...

Grids (13) <dimension family> : <associated variables> 

[1]   D10,D6 : session_id
[2]   D11,D6 : animal_id
[3]   D6,D3  : hold_for_water_zscored_df_over_f, first_bar_off_zscored_df_over_f, first_spout_zscored_df_over_f, hold_for_water_zscored_df_over_f_analog_2, first_bar_off_zscored_df_over_f_analog_2, first_spout_zscored_df_over_f_analog_2    **ACTIVE GRID** ( 5108950  values per variable)
[4]   D7,D6  : uid
[5]   D8,D6  : trigger
[6]   D9,D6  : trial_outcome
[7]   D0     : lick_idx
[8]   D1     : bar_off_idx
[9]   D2     : spout_idx
[10]   D3     : event_time
[11]   D4     : US_end_timer_idx
[12]   D5     : US_random_water_idx
[13]   D6     : valid, Go_to_get_water, US_end_timer, break_after_abort, water_on, water by bar_off, water by spout, water for free, water success, button_press, busy_win_timer, spout, waiting_for_spout, US_jackpot, US_omission, US_random_water, Reward, Extra reward, success, first_reach_travel_time, last_bar_off_time, fir

In [12]:
hyper_vars(xarray_nc)

id,name,type,ndims,natts,dim_coord
<int>,<chr>,<chr>,<int>,<int>,<lgl>
37,hold_for_water_zscored_df_over_f,NC_DOUBLE,2,2,FALSE
38,first_bar_off_zscored_df_over_f,NC_DOUBLE,2,2,FALSE
39,first_spout_zscored_df_over_f,NC_DOUBLE,2,2,FALSE
40,hold_for_water_zscored_df_over_f_analog_2,NC_DOUBLE,2,2,FALSE
41,first_bar_off_zscored_df_over_f_analog_2,NC_DOUBLE,2,2,FALSE
42,first_spout_zscored_df_over_f_analog_2,NC_DOUBLE,2,2,FALSE


In [16]:
nc_grid <- activate(xarray_nc, "D6,D3")
df <- hyper_tibble(nc_grid)

In [29]:
df[,c("first_bar_off_zscored_df_over_f", "event_time", "trial_id")]

first_bar_off_zscored_df_over_f,event_time,trial_id
<dbl>,<chr>,<chr>
0.58949256,-2000,1
-0.42610866,-2000,2
-0.21162241,-2000,3
0.06312187,-2000,4
0.94192533,-2000,11
-1.24835034,-2000,12
-0.67955592,-2000,13
-0.06938034,-2000,14
-0.39262383,-2000,15
